# The Docker Image Construction Problem

[Ryan J. O'Neil](mailto:roneil1@gmu.edu)  
[Systems Engineering & Operations Research](http://seor.gmu.edu)  
George Mason University

## Abstract

The abstract goes here.

## Introduction

## Formulation

## Solution Techniques

### MILP Model

### Heuristic

### Benders Decomposition